In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager

from time import sleep
from datetime import datetime
from random import uniform

import numpy as np
import pandas as pd

In [2]:
categoria = 'trucha'#'mezcla torta' #'cheesecake' #'mezcla pastel' #'postres'
length = 0

iteration = 0
max_iter = 10

# Scraping de almacenes Éxito

In [3]:
def scraping_exito(categoria: object, lugar: object, n_clicks:int = 20):
    '''
    Esta función recibe una palabra clave y la busca en página exito.com. De allí toma el precio, precio
    por gramo y el nombre y retorna los datos de todos los productos que encuentra en un dataframe.
    ------------------------------------------------------------------------------------------------------
    Args:
    categoria: palabra clave que se buscará en la página (object)
    
    lugar: ciudad en la que buscará los productos disponibles (object)
    
    n_clicks: cantidad de veces que pedirá que se refresque la página para ver más productos, por defecto
    lo hará 20 veces (int)
    '''
    search_url='https://www.exito.com/{categoria}?_q={categoria}&map=ft'
    formulario = 'react-select-2-input'
    confirmar = 'exito-geolocation-3-x-primaryButton.shippingaddress-confirmar'
    items = '//div[contains(@class, "galleryItem")]'
    ver_mas = '//div[contains(@class, "buttonShowMore")]//button'

    xpath_precio = './/div[contains(@class, "exito-vtex-components-4-x-PricePDP")]'
    xpath_pgramo = './/div[contains(@class, "exito-vtex-components-4-x-validatePumValue")]'
    xpath_nombre = './/span[contains(@class, "vtex-store-components-3-x-productBrand")]'
    xpath_href = './/a[contains(@class, "clearLink")]'

    precio = []
    pgramo = []
    nombre = []

    driver = webdriver.Chrome(ChromeDriverManager().install())

    driver.get(search_url.format(categoria = categoria))
    sleep(uniform(8.0, 10.0))
    
    ########################################################
    #fragmento del código del día sin IVA 
    ########################################################
    #sleep(uniform(8.0, 10.0))
    #boton = driver.find_element_by_xpath('.//div[contains(@id, "modal-accept-button")]/button')
    #boton.click()
    ########################################################

    ubicacion = driver.find_element_by_id(formulario)
    ubicacion.send_keys(lugar, Keys.RETURN)
    boton = driver.find_element_by_class_name(confirmar)
    boton.click()
    sleep(uniform(10.0, 15.0))
    
    if n_clicks > 0:
        try:
            boton = driver.find_element_by_xpath(ver_mas)
        except:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            boton = driver.find_element_by_xpath(ver_mas)
    for i in range(n_clicks):
        try:
            boton.click()
            sleep(uniform(10.0, 15.0))
            boton = driver.find_element_by_xpath(ver_mas)
        except:
            try:
                driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
                sleep(uniform(10.0, 15.0))
                boton = driver.find_element_by_xpath(ver_mas)
            except:
                break

    anuncios = driver.find_elements_by_xpath(items)
    for anuncio in anuncios:
        try:
            precio.append(anuncio.find_element_by_xpath(xpath_precio).text)
        except:
            precio.append(None)
        
        try:
            pgramo.append(anuncio.find_element_by_xpath(xpath_pgramo).text)
        except:
            pgramo.append(None)
            
        try:
            nombre.append(anuncio.find_element_by_xpath(xpath_nombre).text)
        except:
            nombre.append(None)
        
    driver.quit()
    
    arr_lugar = np.repeat(lugar, len(nombre))
    
    fecha = datetime.today().strftime('%Y-%m-%d')
    arr_fecha = np.repeat(fecha, len(nombre))
    
    dict_df = {'nombre': nombre,
               'precio': precio,
               'precio_por_gramo': pgramo,
               'fecha': arr_fecha,
               'lugar': arr_lugar,
                }
    df = pd.DataFrame(dict_df)
    
    return df


def buscar_terminos_ciudad(categoria:object, 
                           lugares:list = ['Bogotá', 'Medellín', 'Barranquilla', 'Cali', 'Cartagena', 
                                           'Armenia', 'Bello', 'Bucaramanga', 'Chía', 'Cúcuta',
                                           'Fusagasuga', 'Ibague', 'La Ceja', 'Manizales', 'Monteria',
                                           'Neiva', 'Pasto', 'Pereira', 'Popayán', 'Rionegro', 
                                           'Santa Marta', 'Sincelejo', 'Soacha', 'Tunja', 'Valledupar', 
                                           'Villavicencio', 'Zipaquira'], 
                           n_clicks:int = 20):
    '''
    Esta función recibe una palabra clave y unas ciudades, y la busca en página exito.com para esas
    ciudades. De allí toma el precio, precio por gramo y el nombre y retorna los datos de todos los 
    productos que encuentra en un dataframe, y un arreglo con todas las ciudades con las que no pudo 
    completar el proceso.
    ------------------------------------------------------------------------------------------------------
    Args:
    categoria: palabra clave que se buscará en la página (object)
    
    lugares: ciudades en las que buscará los productos disponibles, por defecto lo hará para todas las
    ciudades (object)
    
    n_clicks: cantidad de veces que pedirá que se refresque la página para ver más productos, por defecto
    lo hará 20 veces (int)
    '''
    errores = []

    df_final = pd.DataFrame(columns = ['nombre', 'precio', 'precio_por_gramo', 'fecha', 'lugar'])
    for lugar in lugares:
        try:
            df = scraping_exito(categoria, lugar, n_clicks)
            df_final = df_final.append(df, ignore_index = True)
            sleep(uniform(10.0, 15.0))
        except:
            errores.append(lugar)
            driver = webdriver.Chrome(ChromeDriverManager().install())
            driver.quit()
            
    return df_final, errores

In [4]:
res = buscar_terminos_ciudad(categoria, n_clicks = 0)
df_final = res[0].copy()
df_final.to_csv('scraping_de_exito_de_{}.csv'.format(categoria), index = False)

while len(res[1]) > length:
    res = buscar_terminos_ciudad(categoria, res[1], n_clicks = 0)
    df_final = df_final.append(res[0], ignore_index = True)
    df_final.to_csv('scraping_de_exito_de_{}.csv'.format(categoria), index = False)
    
    iteration+=1
    if iteration > max_iter:
        print('Se alcanzó la máxima cantidad de iteraciones')
        break
    
    
print('''
Hemos terminado!

Se ha descargado la informacion de {} productos
'''.format(df_final.shape[0]))



====== WebDriver manager ======
Could not get version for google-chrome.Is google-chrome installed?
Get LATEST chromedriver version for None google-chrome
Driver [C:\Users\david\.wdm\drivers\chromedriver\win32\103.0.5060.134\chromedriver.exe] found in cache
C:\Users\david\AppData\Local\Temp\ipykernel_11224\3914614200.py:29: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
C:\Users\david\AppData\Local\Temp\ipykernel_11224\3914614200.py:42: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  ubicacion = driver.find_element_by_id(formulario)
C:\Users\david\AppData\Local\Temp\ipykernel_11224\3914614200.py:44: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  boton = driver.find_element_by_class_name(confirmar)
C:\Users\david\AppData\Local\Temp\ipykernel_11224\39146142



====== WebDriver manager ======
Could not get version for google-chrome.Is google-chrome installed?
Get LATEST chromedriver version for None google-chrome
Driver [C:\Users\david\.wdm\drivers\chromedriver\win32\103.0.5060.134\chromedriver.exe] found in cache



Hemos terminado!

Se ha descargado la informacion de 199 productos



# Scraping de almacenes Jumbo

In [6]:
search_url='https://www.tiendasjumbo.co/trucha?_q=trucha&map=ft&page=1'
black_dom_selector = "return document.querySelector('impulse-search').shadowRoot.querySelectorAll('div.content')"

nombre = []
marca = []
pcon_descuento = []
psin_descuento = []
pgramo = []

driver = webdriver.Chrome(ChromeDriverManager().install())

driver.get(search_url)
sleep(uniform(8.0, 10.0))

driver.execute_script(black_dom_selector)



====== WebDriver manager ======
Could not get version for google-chrome.Is google-chrome installed?
Get LATEST chromedriver version for None google-chrome
Driver [C:\Users\david\.wdm\drivers\chromedriver\win32\103.0.5060.134\chromedriver.exe] found in cache
C:\Users\david\AppData\Local\Temp\ipykernel_11224\563539970.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


JavascriptException: Message: javascript error: Cannot read properties of null (reading 'shadowRoot')
  (Session info: chrome=103.0.5060.134)
Stacktrace:
Backtrace:
	Ordinal0 [0x00685FD3+2187219]
	Ordinal0 [0x0061E6D1+1763025]
	Ordinal0 [0x00533E78+802424]
	Ordinal0 [0x00536854+813140]
	Ordinal0 [0x00536712+812818]
	Ordinal0 [0x0053708A+815242]
	Ordinal0 [0x0058D9F9+1169913]
	Ordinal0 [0x0057CB8C+1100684]
	Ordinal0 [0x0058CFC2+1167298]
	Ordinal0 [0x0057C9A6+1100198]
	Ordinal0 [0x00556F80+946048]
	Ordinal0 [0x00557E76+949878]
	GetHandleVerifier [0x009290C2+2721218]
	GetHandleVerifier [0x0091AAF0+2662384]
	GetHandleVerifier [0x0071137A+526458]
	GetHandleVerifier [0x00710416+522518]
	Ordinal0 [0x00624EAB+1789611]
	Ordinal0 [0x006297A8+1808296]
	Ordinal0 [0x00629895+1808533]
	Ordinal0 [0x006326C1+1844929]
	BaseThreadInitThunk [0x75D36739+25]
	RtlGetFullPathName_UEx [0x776F8FEF+1215]
	RtlGetFullPathName_UEx [0x776F8FBD+1165]


In [12]:
def scraping_jumbo(categoria: object, pagina: int, max_len:int = 7):
    '''
    Esta función recibe una palabra clave y la busca en página tiendasjumbo.co. De allí toma 
    el precio con y sin descuento, el nombre, la marca y el precio por gramo.
    Retorna los datos de todos los productos que encuentra en un dataframe.
    ------------------------------------------------------------------------------------------------------
    Args:
    categoria: palabra clave que se buscará en la página (object)
    
    pagina: página del dom en el que buscará (int)
    
    max_len: parámetro interno que permite distinguir si el producto está en descuento o no (int)
    '''
    search_url='https://www.tiendasjumbo.co/{categoria}?_q={categoria}&map=ft&page={pagina}'
    black_dom_selector = "return document.querySelector('impulse-search').shadowRoot.querySelectorAll('div.content')"

    nombre = []
    marca = []
    pcon_descuento = []
    psin_descuento = []
    pgramo = []

    driver = webdriver.Chrome(ChromeDriverManager().install())

    driver.get(search_url.format(categoria = categoria, pagina = pagina))
    sleep(uniform(8.0, 10.0))

    items = driver.execute_script(black_dom_selector)
    if len(items) == 0:
        driver.quit()
        raise ValueError('Ya no hay más qué buscar')

    for item in items:
        features = item.text.split('\n')
        try:
            nombre.append(features[0])
        except:
            nombre.append(None)
        
        try:
            marca.append(features[1])
        except:
            marca.append(None)
    
        if len(features) > max_len:
            try:
                psin_descuento.append(features[2])
            except:
                psin_descuento.append(None)
            
            try:
                pcon_descuento.append(features[3])
            except:
                pcon_descuento.append(None)
            
            try:
                pgramo.append(features[4])
            except:
                pgramo.append(None)
        else:
            pcon_descuento.append(None)
            
            try:
                psin_descuento.append(features[2])
            except:
                psin_descuento.append(None)
            
            try:
                pgramo.append(features[3])
            except:
                pgramo.append(None)


    driver.quit()

    fecha = datetime.today().strftime('%Y-%m-%d')
    arr_fecha = np.repeat(fecha, len(nombre))
    
    dict_df = {'nombre': nombre,
               'marca': marca,
               'precio_sin_descuento': pcon_descuento,
               'precio_con_descuento': psin_descuento,
               'precio_por_gramo': pgramo,
                'fecha': arr_fecha
        }
    df = pd.DataFrame(dict_df)

    return df

def buscar_termino(categoria:object):
    '''
    Esta función busca el término hasta que no haya más productos relacionados a él en la página de almacenes
    Jumbo
    ------------------------------------------------------------------------------------------------------
    Args:
    categoria: palabra clave que se buscará en la página (object)
    '''
    pag = 1
    df_final = pd.DataFrame(columns = ['nombre', 'marca', 'precio_sin_descuento', 'precio_con_descuento', 'precio_por_gramo', 'fecha'])
    while True:
        try:
            df = scraping_jumbo(categoria, pag)
            df_final = df_final.append(df, ignore_index = True)
            pag += 1
        except:
            print('''
            Hemos terminado!
            
            Se ha descargado la informacion de {} productos
            '''.format(df_final.shape[0]))
            break
        
    return df_final

In [13]:
df_final = buscar_termino(categoria)
df_final.to_csv('scraping_de_jumbo_de_{}.csv'.format(categoria), index = False)



====== WebDriver manager ======
Could not get version for google-chrome.Is google-chrome installed?
Get LATEST chromedriver version for None google-chrome
Driver [C:\Users\david\.wdm\drivers\chromedriver\win32\103.0.5060.134\chromedriver.exe] found in cache
C:\Users\david\AppData\Local\Temp\ipykernel_17904\2071218949.py:23: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())



            Hemos terminado!
            
            Se ha descargado la informacion de 0 productos
            


# Scraping de almacenes Euro

In [10]:
def scraping_euro(categoria: object, pagina: int):
    '''
    Esta función recibe una palabra clave y la busca en página eurosupermercados.com.co. De allí toma 
    el precio y el nombre.
    Retorna los datos de todos los productos que encuentra en un dataframe.
    ------------------------------------------------------------------------------------------------------
    Args:
    categoria: palabra clave que se buscará en la página (object)
    
    pagina: página del dom en el que buscará (int)
    '''
    search_url='https://www.eurosupermercados.com.co/catalogsearch/result/index/?p={pagina}&q={categoria}'
    items = '//div[contains(@class, "product details product-item-details")]'
    
    precio = []
    pgramo = []
    nombre = []
    
    driver = webdriver.Chrome(ChromeDriverManager().install())

    driver.get(search_url.format(categoria = categoria, pagina = pagina))
    sleep(uniform(8.0, 10.0))

    anuncios = driver.find_elements_by_xpath(items)
    for anuncio in anuncios:
        
        lista_anuncio = anuncio.text.split('\n')
        try:
            precio.append(lista_anuncio[0])
        except:
            precio.append(None)
            
        try:
            nombre.append(lista_anuncio[1])
        except:
            nombre.append(None)
                
    driver.quit()

    fecha = datetime.today().strftime('%Y-%m-%d')
    arr_fecha = np.repeat(fecha, len(nombre))
    
    dict_df = {'nombre': nombre,
               'precio': precio,
                'fecha': arr_fecha
        }
    df = pd.DataFrame(dict_df)

    return df

def buscar_producto(categoria:object, pag_max: int):
    '''
    Esta función busca el término por la cantidad de páginas que el usuario indique. Busca en la página de
    Euro
    ------------------------------------------------------------------------------------------------------
    Args:
    categoria: palabra clave que se buscará en la página (object)
    '''
    df_final = pd.DataFrame(columns = ['nombre', 'precio', 'fecha'])
    for pag in range(1, pag_max+1):
        df = scraping_euro(categoria, pag)
        df_final = df_final.append(df, ignore_index = True)
            
    print('''
    Hemos terminado!
    
    Se ha descargado la informacion de {} productos
    '''.format(df_final.shape[0]))
        
    return df_final

In [11]:
df_final = buscar_producto(categoria, 15)
df_final.to_csv('scraping_de_euro_de_{}.csv'.format(categoria), index = False)



====== WebDriver manager ======
Could not get version for google-chrome.Is google-chrome installed?
Get LATEST chromedriver version for None google-chrome
Driver [C:\Users\david\.wdm\drivers\chromedriver\win32\103.0.5060.134\chromedriver.exe] found in cache
C:\Users\david\AppData\Local\Temp\ipykernel_17904\397378575.py:19: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
C:\Users\david\AppData\Local\Temp\ipykernel_17904\397378575.py:24: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  anuncios = driver.find_elements_by_xpath(items)
C:\Users\david\AppData\Local\Temp\ipykernel_17904\397378575.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_final = df_final.append(df, ignore_index = True)


====== WebDriver manager ======
C


    Hemos terminado!
    
    Se ha descargado la informacion de 90 productos
    
